In [1]:
import plotly.express as px
import pandas as pd

In [2]:
prices = pd.read_csv("price data.csv")
fields = pd.read_csv("labels.csv")

prices.set_index("Date", inplace = True)

In [3]:
sigma = prices.pct_change().rolling(252).std()*(252**(1/2))
delta = prices.pct_change(252)

sigma.drop(sigma[sigma.isnull().all(1)].index, inplace = True)
delta.drop(delta[delta.isnull().all(1)].index, inplace = True)

df = pd.concat([delta.stack(), sigma.stack()], axis = 1)
df.rename(columns = {0:"Percent Total Return", 1:"Percent Standard Deviation"}, inplace = True)
df.index.rename(('Date','Ticker'),inplace = True)

df.reset_index(level = "Date", inplace=True)
df = df.merge(right = fields, how = 'left', on = 'Ticker')
df['Size'] = 1
df['Percent Total Return'] *= 100
df['Percent Standard Deviation'] *= 100

In [8]:
fig = px.scatter(df, x="Percent Standard Deviation", y="Percent Total Return", animation_frame="Date", animation_group="Ticker",
           color="Type", hover_name="Name", title = "Multi Asset Class Historical 1 Year Risk & Return Characteristics",
           size = "Size", size_max=10, range_x=[0,75], range_y=[-100,100])
fig

In [7]:
fig.write_html("riskreturn.html")